Аня Александрова
бкл-192

#Задание: организовать хранение и поиск метаинформации о документах: автор, дата создания документа, кто создал, историей правок (дата, кто правил) и утверждения (дата, резолюция, чья резолюция). Хранение организовать в формате json.#

##Первым делом создаём воображаемую базу данных метаинформации о воображаемых документах. Был соблазн сразу написать json-строку, но это, кажется, не совсем честно. И не совсем просто. Поэтому пишем код, который позволяет нам из экселевской таблицы, сохранённой в формате .csv делать строку .json##

In [315]:
import json, csv

meta_data = {}

with open('C:\\Users\\anuta\\info.csv', encoding='utf-8') as csv_file:
    with open('C:\\Users\\anuta\\good_info.json', 'w', encoding='utf-8') as json_file:
        csv_read = csv.reader(csv_file)
        for row in csv_read:
            for i in row:
                doc_num, version_num, latest_version_num, author, creation_date, corrector, correction_date, resolution_date, resolution, resolutioner = i.split(';')
                unique_num = doc_num+version_num+latest_version_num
                unique_num_doc = {unique_num:{'номер документа':doc_num, 'номер версии':version_num, 'номер актуальной версии':latest_version_num, 'автор':author, 'дата создания':creation_date, 'автор правки':corrector, 'дата правки':correction_date, 'дата резолюции':resolution_date, 'резолюция':resolution, 'резолюционер':resolutioner}}
                meta_data.update(unique_num_doc)          
        json_meta_data = (json.dump(meta_data, json_file, ensure_ascii=False))

##А так как будто бы даже можно добавить метаинформацию о новом документе без особых проблем.

Итак, получился Словарь словарей. Каждый словарь состоит из одного ключа (== уникального номера версии документа) и значения-словаря, который в свою очередь состоит из пар с собственно метаинформацией о документе. Уникальный номер версии документа - последовательность из номера документа по порядку, номера данной версии документа и номера последней версии документа. Технически уникальный номер может состоять только из номера документа по порядку и номера данной версии документа, но мне кажется, что удобно, когда по номеру можно понять, актуальна ли читаемая в данный момент версия и, если нет, как найти актуальную (== какой номер а последней версии документа)##

In [316]:
with open('C:\\Users\\anuta\\good_info.json', 'r', encoding='utf-8') as filename:
    data = json.load(filename)
    for key, value in data.items():
        print(key, value, '\n')

111 {'номер документа': '1', 'номер версии': '1', 'номер актуальной версии': '1', 'автор': 'Ю. А. Ландер', 'дата создания': '01.09.2020', 'автор правки': '', 'дата правки': '', 'дата резолюции': '01.09.2020', 'резолюция': 'навсегда убрать печень из комплекса на басманной', 'резолюционер': 'Ю. А. Ландер'} 

212 {'номер документа': '2', 'номер версии': '1', 'номер актуальной версии': '2', 'автор': 'Э. С. Клышинский', 'дата создания': '25.10.2012', 'автор правки': '', 'дата правки': '', 'дата резолюции': '26.10.2012', 'резолюция': 'надо давать детям конфеты за правильные ответы', 'резолюционер': 'Ю. А. Ландер'} 

222 {'номер документа': '2', 'номер версии': '2', 'номер актуальной версии': '2', 'автор': 'Э. С. Клышинский', 'дата создания': '25.10.2012', 'автор правки': 'О. С. Сериков', 'дата правки': '03.03.2015', 'дата резолюции': '04.03.2015', 'резолюция': 'надо давать конфеты не только за правильные ответы но и за правильные вопросы', 'резолюционер': 'Ю. А. Ландер'} 

311 {'номер докуме

###Как найти информацию о версии документа по её уникальному номеру? Тут всё предельно просто:###

In [317]:
def unique_num_searcher(unique_num):
    print('Документ номер', unique_num, '\n')
    for meta, info in data[unique_num].items():
        print(meta, '-', info)
    #print(data[unique_num])

In [318]:
unique_num_searcher('511')

Документ номер 511 

номер документа - 5
номер версии - 1
номер актуальной версии - 1
автор - А. А. Александрова
дата создания - 29.09.2020
автор правки - 
дата правки - 
дата резолюции - 29.09.2020
резолюция - математический анализ надо отменить
резолюционер - А. А. Александрова


###Как найти информацию о версии документа по кусочку метаинформации, которая содержится в словаре версии?###

In [331]:
def meta_data_searcher(request):
    
    for unique_num, meta_data in data.items():
        #print(unique_num, meta_data)
        for meta, info in meta_data.items():
            if request == info:
                print('Ваш запрос содержится в метаинформации о документе номер', unique_num)
                for name, cont in meta_data.items():
                    if not cont == '':
                        print(name, '-', cont)
                print('\n')
            

###Попробуем найти что-то от Ю. А. Ландера###

In [332]:
meta_data_searcher('Ю. А. Ландер')   

Ваш запрос содержится в метаинформации о документе номер 111
номер документа - 1
номер версии - 1
номер актуальной версии - 1
автор - Ю. А. Ландер
дата создания - 01.09.2020
дата резолюции - 01.09.2020
резолюция - навсегда убрать печень из комплекса на басманной
резолюционер - Ю. А. Ландер


Ваш запрос содержится в метаинформации о документе номер 111
номер документа - 1
номер версии - 1
номер актуальной версии - 1
автор - Ю. А. Ландер
дата создания - 01.09.2020
дата резолюции - 01.09.2020
резолюция - навсегда убрать печень из комплекса на басманной
резолюционер - Ю. А. Ландер


Ваш запрос содержится в метаинформации о документе номер 212
номер документа - 2
номер версии - 1
номер актуальной версии - 2
автор - Э. С. Клышинский
дата создания - 25.10.2012
дата резолюции - 26.10.2012
резолюция - надо давать детям конфеты за правильные ответы
резолюционер - Ю. А. Ландер


Ваш запрос содержится в метаинформации о документе номер 222
номер документа - 2
номер версии - 2
номер актуальной верс